## Deep Neural Network For MNIST Classification

#### Import relevant packages

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

#### Data


In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_sample = 0.1*mnist_info.splits['train'].num_examples
num_validation_sample = tf.cast(num_validation_sample, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image/= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_sample)

train_data = shuffled_train_and_validation_data.skip(num_validation_sample)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_sample)

test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

#### Model


##### Outline the model

In [5]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape= (28,28,1)),
            tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
            tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
            tf.keras.layers.Dense(output_size, activation= 'softmax')
])

C:\Users\Williams\anaconda3\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Choose the Optimizer and the Loss Function

In [6]:
model.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

#### Training

In [7]:
NUM_EPOCHS = 5
early_stopping = tf.keras.callbacks.EarlyStopping()
model.fit(train_data, 
          epochs= NUM_EPOCHS,
          callbacks = [early_stopping],
          validation_data= (validation_inputs,validation_targets), 
          verbose= 2)

Epoch 1/5
540/540 - 4s - 8ms/step - accuracy: 0.8823 - loss: 0.4194 - val_accuracy: 0.9380 - val_loss: 0.2185
Epoch 2/5
540/540 - 2s - 4ms/step - accuracy: 0.9451 - loss: 0.1878 - val_accuracy: 0.9510 - val_loss: 0.1727
Epoch 3/5
540/540 - 2s - 4ms/step - accuracy: 0.9573 - loss: 0.1444 - val_accuracy: 0.9620 - val_loss: 0.1319
Epoch 4/5
540/540 - 2s - 3ms/step - accuracy: 0.9655 - loss: 0.1152 - val_accuracy: 0.9657 - val_loss: 0.1200
Epoch 5/5
540/540 - 2s - 3ms/step - accuracy: 0.9706 - loss: 0.0968 - val_accuracy: 0.9717 - val_loss: 0.0973


#### Test the Model

In [8]:
test_loss, test_accuracy= model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step - accuracy: 0.9684 - loss: 0.1059


In [9]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 96.84%
